# Beautiful day, isn't it?

Obvserving shifts in Global and Local Temperature since 1750

## Outline
* Extracting Data from a SQL database
* Importing and reshaping a dataset for analysis
* Calculating a moving average
* Creating and interpreting a simple data visualization 

### 1)Extracting data from a SQL database:
##### QUERY.sql
The data that we are interested in is contained in two tables. We can pull all of the relevant information in a single SQL query by performing a union of the desired records from each of the tables.
*Note, since a union requires output columns to match on both tables, we will create the 'city' column that is missing in the global table. 

*Also note: We are not actually connected to the database here, nor are we incorporating SQL functionality in this notebook, so the following query is for reference purposes only.

In [ ]:
#SELECT year, city, avg_temp
#FROM city_data
#WHERE city = 'Baltimore'
#UNION ALL
#SELECT year, 'global' AS city, avg_temp 
#FROM global_data

We have been provided an easy-access tool which allows us to export the data returned by our query as a .csv file, RESULTS.csv

### 2) Importing the data into Python and reshaping it for analysis
##### format.py

In [ ]:
import pandas as pd

# Reading the data from the CSV file that our SQL query generated into a Pandas DataFrame
raw = pd.read_csv('RESULTS.csv')

# Taking a look:
print(raw.head())
print(raw.tail())

We now have the data we need in a Pandas DataFrame. Let's make it better by reshaping it so that local and global temp are both contained within each observation, indexed by year.  

In [ ]:
# Reshaping the Dataframe 
shaped = pd.pivot_table(raw, values = 'avg_temp', index= 'year', columns= 'city')

#Take another look:
print(shaped.head(10))
print(shaped.tail(10))

We now have our global and local temperatures in their own separate columns, and the data is indexed by year. 

There is missing data at the head and tail of the dataframe and many are missing in both columns. We can safely ignore those years where data is missing since there are so few, so lets trim the dataset.

In [ ]:
#trimming the start and end years where data is missing 
almost_tidy = shaped.loc[1750:2013]

#making sure it worked
print(almost_tidy.head(5))
print(almost_tidy.tail(5))

#check for and locate any other missing values
print(almost_tidy.isnull().any())
print(almost_tidy[almost_tidy.isnull().any(axis=1)])

Great, our head and tail are clean, and we only have one missing value. Let's look more closely to see if that value could be easily imputed: 

In [ ]:
almost_tidy.loc[1775:1785]['Baltimore']

This is an odd result. Not only is the value missing in 1780, it is lower than usual in 1778 and extremely low in 1780. Because of this I don't want to backfill or forward fill. I also don't want to use a measure of centrality because of the presence of outliers. What I will do is use a hybrid approach, replacing the temp at 1780 with the temp from 1778, creating a sort of "V" shape in the data. 

This is an admittedly crude and unscientific approach, but since we're already planning to use moving averages, this shouldn't taint our final analysis. We're going to leave in the outlier because we have no way of telling (for now) whether it is a mistake or if there was some sort of strange weather phenomenon that year in Baltimore. This would be a question for further research!

(we're finally done cleaning our data, so let's rename it too)

In [ ]:
almost_tidy.loc[1780]['Baltimore'] = almost_tidy.loc[1778]['Baltimore']

tidy = almost_tidy.loc[:]
del almost_tidy

### 3) Calculating the moving average
##### analyze.py

Data can be 'smoothed' by calculating a moving average. This reduces the amplitude of unit-wise fluctuations by replacing actual observation values with the average of multiple observation values over a certain range. 

We are going to do this in python by writing a function which will allow us to define how large of a range we want to use to calculate our averages. A higher range or 'width' will produce a 'smoother' output. 

In [ ]:
import numpy as np
 
def run_avg(data, width, bfill = True):
    output = []
    if width > len(data)/2:
        return print("Width too large. Please select a value less than " \
                     + str(len(data)/4) + ".")
            #^^^ This is a safeguard against selecting a range that is too large.
    for i in range(0,(len(data))):
        if i < width-1:
            if bfill:
                output.append(round(np.mean(data[0:width-1]), 2))
            # Backfills with first calculation so output data matches length of input data.
            continue
        else:
            output.append(round(np.mean(data[i-width+1:i]), 2))
    return np.array(output)
    # Returns running average as ndarray

Now that we have a way of calculating moving averages on a series, lets apply this function to the data in our dataframe. We will do this for the local temp data and the global temp data, and then will add these moving averages to our existing dataframe to keep everything in one place. For this exercise, we will be using a 10-year moving average. 

In [ ]:
local_avg_10 = run_avg(tidy['Baltimore'], 10)
global_avg_10 = run_avg(tidy['global'], 10)

tidy.loc[:, 'Balt_avg'] = local_avg_10
tidy.loc[:, 'global_avg'] = global_avg_10

#as usual, let's take a look:
print(tidy[4:16])


Awesome!

### 4) Creating and interpreting a simple data visualization

Now that we have all of our data in its final format, we want to explore it visually. 

Before we look at trends, I'd like to just check and see how much of an effect our moving average had on how the data looks. 

In [ ]:
import matplotlib.pyplot as plt

tidy.plot(subplots=True, lw=2, title='Smoothed vs Unsmoothed trendlines')
plt.show()

We can see, even just at a glance, that our moving average had the intended effect of 'smoothing' the data. We also see that our outlier at 1778 has a pretty significant effect on the moving average. Actually, using a moving average can, in a way, strengthen the effect of an outlier because that outlier will be used in multiple average calculations rather than just in a single observation! We would want to address this if we were going to proceed further with our analysis.

Lets look more closely at the smoothed data.

In [ ]:
tidy[['Balt_avg','global_avg']].plot(lw=4, title='Temperature 1750-2013 \
(10-year moving avg)').set_ylabel('Temperature')
plt.show()

From this, we can observe the following: <br/>
1) The temperature in Baltimore has been significantly higher than the global average temperature for at least the past 250 years. This makes sense because Baltimore is relatively close to the equator. <br/>
2) Temperatures in Baltimore and in the world on average have been increasing. While this seems pretty clear, we can verify by plotting lines of best fit. 

In [ ]:
#Calculating the slope and y-intercept of each line of best fit. 
mbalt, bbalt = np.polyfit(tidy.index, tidy['Balt_avg'], 1) 
mglob, bglob = np.polyfit(tidy.index, tidy['global_avg'], 1)

#superimposing the linear regression onto the data plot
tidy[['Balt_avg','global_avg']].plot(lw=4, title='Temperature 1750-2013 \
(10-year moving avg)').set_ylabel('Temperature')
plt.plot(tidy.index, mglob*tidy.index+bglob, label='global', lw=2)
plt.plot(tidy.index, mbalt*tidy.index+bbalt, label='Baltimore', lw=2)
plt.show()

As this demonstrates, the temperature is trending upwards for both local and global data. This also brings us to:<br/>
3) The Baltimore line is steeper than the global line which means that avg temperature has been increasing more rapidly for Baltimore than it has for the world as a whole (at least during the years between 1750 and 2013) <br/>
4) The final (and perhaps most interesting, at least for someone who is interested in Baltimore!) observation is that, for the most part, the general trends of the temperatures in Baltimore closely matches that of the world, but about 3 degrees hotter. 